# Import Libraries

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import numpy as np
import pandas as pd
import tqdm
import yaml
import glob
import os

In [3]:
from src.components.feature_extraction import feature_extraction
from src.utils import get_root_directory

In [4]:
# Get root directory of the project
root_dir = get_root_directory()

In [5]:
# Maximize Column Display 
pd.set_option('display.max_colwidth', None)     # Display all content within each cell without truncation
pd.set_option('display.max_columns', None)      # Display all columns
pd.set_option('display.width', None)            # Display entire width of DataFrame is displayed

pd.set_option('display.max_rows', None)         # Display all rows

# Extract Traffic Signal Profile

In [6]:
# Configurations
signal_id = '1080'
day = 19; month = 9; year = 2024 

In [7]:
# Extract Traffic Signal Profile
traffic_signal_profile = feature_extraction.TrafficSignalProfile(day=day, 
                                                                 month=month, 
                                                                 year=year)

df_vehicle_phase_profile_id = traffic_signal_profile.extract_vehicle_phase_profile(signal_id=signal_id)
df_vehicle_cycle_profile_id = traffic_signal_profile.extract_vehicle_cycle_profile(signal_id=signal_id)

In [8]:
# Extract signal features
signal_feature_extract = feature_extraction.SignalFeatureExtract(day=day, 
                                                                 month=month, 
                                                                 year=year)
df_spat_id = signal_feature_extract.spat(signal_id=signal_id) 

In [9]:
# Extract traffic features
traffic_feature_extract = feature_extraction.TrafficFeatureExtract(day=day, 
                                                                   month=month,
                                                                   year=year)
df_volume_id = traffic_feature_extract.volume(signal_id=signal_id)